## I. Setup, Installations, and Imports

#### (a) Prereqs

1. You have jupyterlab running in your base conda environment. 
1. Create the `nlp_stanza` environment via the readme's instructions. 
1. Add that environment's python kernel to jupyterlab via the following code. This will also download the `spacy` model. 

    ```terminal
    $ conda activate nlp_stanza          
    (nlp_stanza)$ conda install ipykernel
    (nlp_stanza)$ python -m ipykernel install --user --name=nlp_stanza --display-name "NLP Stanza Env"
    (nlp_stanza)$ python -m spacy download en_core_web_sm
    (nlp_stanza)$ conda deactivate
    $ jupyter lab          
    ```
    
**Make sure this script is being run with the nlp_stanza conda environment's kernel. In the upper right corner, click the kernel name and change to "NLP Stanza Env".**


#### (b) Installations

Run these if not on your computer already.

In [1]:
### NOTE: THESE ARE REDUNDANT IF PREREQS ABOVE MEET
# ! pip install spacy
# ! pip install nltk
# ! python -m spacy download en_core_web_sm
# ! pip install svgling
# ! python -m pip install textacy
# ! pip install stanza

#### (c) Imports

In [2]:
import os
import stanza
# stanza.download('en') # This downloads the English models for the neural pipelin
stanza_nlp = stanza.Pipeline('en', download_method=None) # This sets up a default neural pipeline in English

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
from __future__ import unicode_literals
import spacy

spacy_nlp = spacy.load("en_core_web_sm",)  

nltk.download('words')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')
nltk.download('state_union')

C:\Users\DonsLaptop\anaconda3\envs\nlp_stanza\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-30 14:14:02 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

2023-03-30 14:14:02 INFO: Using device: cpu
2023-03-30 14:14:02 INFO: Loading: tokenize
2023-03-30 14:14:02 INFO: Loading: pos
2023-03-30 14:14:03 INFO: Loading: lemma
2023-03-30 14:14:03 INFO: Loading: constituency
2023-03-30 14:14:04 INFO: Loading: depparse
2023-03-30 14:14:04 INFO: Loading: sentiment
2023-03-30 14:14:04 INFO: Loading: ner
2023-03-30 14:14:05 INFO: Done loading processors!

True

## II. Defined Functions Used In Program


#### (a) Named Entity Recognizer Function
**Input:** A sentence <br> 
**Output:** A dictionary in the form of: <br>
&emsp;**{'entities':** [list of named entity recognizers of that sentence], **'verbs':** [list of the verbs], **'subjects':** [list of the subjects]**, 'subj_verb_linkages',** [root subject verb linkage array]**}**

In [6]:
def ner(sentence): 
    
    entities = []
    verbs = []
    subjects = []
    subj_verb_linkages = []
    
    #Find the entities in the sentence
    words  = nltk.word_tokenize(sentence)        # break down the sentence into words
    tagged = nltk.pos_tag(words)                 # tag the words with Part of Speech 
    chunks = nltk.ne_chunk(tagged, binary=False) # binary = False named entities are classified (i.e PERSON, ORGANIZATION)
    
    for chunk in chunks:
        if hasattr(chunk, 'label'):              # hasattr(obj, key) -- checking if chunks have a label or not 
            entities.append(' '.join(c[0] for c in chunk)) # append entities to array
    
    
    #Find the verbs/subjects in the sentence
             # load in the spacy model
    doc = spacy_nlp(sentence)                          # create spacy doc object
    
    verbs = [token.text for token in doc if token.pos_ == "VERB"]     # traverse thru the tokens, find the verbs
    subjects = [token.text for token in doc if token.dep_ == "nsubj"]  # traverse thru the tokens, find the subjects
    
    
    #Find the Root Subject-verb linkages in the sentences using stanza
    doc = stanza_nlp(sentence)
     
    subject, root_verb = None, None
    
    for word in doc.sentences[0].words:    # for each word in the sentence
        if word.deprel == 'root':          # if a word is the root, its dependency relation label is 'root'. thus if this is true, the curr word = root word

            root_verb = word.text          # save the root verb  
            root_id = word.id              # get the words id 
            
            for w in doc.sentences[0].words:                      # loop over words in the sentence
                if w.head == root_id and w.deprel == 'nsubj':     # if words head attribute = root_id, then its a a direct dependent of the root (is a child of the root)
                    subject = w.text 
    
    if subject and root_verb: # not empty
        subj_verb_linkages = [subject, root_verb]   # subj_verb linkages array 
    
        
    return {'entities':entities, 
            'verbs':verbs,
            'subjects':subjects,
            'subj_verb_linkages':subj_verb_linkages} 

#### (b) Filename Traversal Function
**Input:** A Filename **(i.e /inputs/ex10.txt)** <br>
**Output:** A dict of dicts: <br>
&emsp;&emsp;&emsp;&emsp;**{sentence: {sentence_level_outputs}}** <br>
&emsp;&emsp;where sentence_level_outputs <br>
&emsp;&emsp;&emsp;&emsp;**{'analysis type/function' : output thereof}**


In [7]:
def doc_trawl(filename):

    file_output = {}
    
    with open(filename, "r") as fp:
        raw = BeautifulSoup(fp.read(), 'html.parser').get_text()
        raw_sentences = nltk.sent_tokenize(raw)
    
    for sentence in raw_sentences:
        
        # put all output of this sentence here 
        # key=analysis type/function, value=output thereof
        sentence_level_outputs = {} 
        
        # use ner function  
        sentence_level_outputs.update(ner(sentence))
        
        # any other output we want to add that doesn't rely on the ner tokenization
        # should be done here
        # to show that the plumbing works correctly, let's just add a rand variable:
        sentence_level_outputs['random_num'] = np.random.uniform()
        
        # Add to output dictionary
        file_output.update({sentence:sentence_level_outputs})
        
    return file_output

## III. Automation

In [8]:
file_sentence_dict = {}
files = glob.glob("inputs/*") #get all the files in the inputs folder

for file in tqdm(files,total=len(files)):
    file_sentence_dict.update({file: doc_trawl(file)}) #update the dictionary 

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [04:43<00:00, 141.51s/it]


## IV. Unpacking that into DF

Dataframe with 
- index is filename-sentence
- columns are sentence level variables

Now we can do diagnostics, examine the output, and use it faster!

In [12]:
def unpack_tri_level_dict(a_dict):
    df = pd.concat(map(lambda x: pd.DataFrame.from_dict(x).T, a_dict.values()), keys=a_dict.keys())
    df.index = df.index.rename(['file','sentence'])
    return df

display(unpack_tri_level_dict(file_sentence_dict))

os.makedirs('outputs',exist_ok=True)

unpack_tri_level_dict(file_sentence_dict).to_csv('outputs/test.csv',index=True)

entities  \
file               sentence                                                                                                
inputs\ex10-11.txt \nEX-10\n2\nex10-11.txt\nCFC INTERNATIONAL, INC...  [CFC, INC., CONTRACT, AGREEMENT, DATED, CFC In...   
                   1.                                                                                                 []   
                   General Provisions\n----------------------\n\nA...                                                 []   
                   This\naddendum applies to CFC products B10EK bl...                              [CFC, B10EK, B5603AB]   
                   2.0  Distribution\n-----------------\n\nBuyer a...             [Buyer, Baxter Healthcare Corporation]   
...                                                                                                                  ...   
inputs\ex10.txt    13.6 This Agreement is deemed to have been ente...                               [Illinois, Illinois]   
                   13.7 In the event that a court of competent jur...                                                 []   
                   In Witness Whereof, the parties have caused thi...                                  [Witness Whereof]   
                   BAXTER HEALTHCARE CORP             CFC INTERNAT...  [BAXTER, HEALTHCARE, CORP, CFC, Dave Valentini...   
                   Purchasing                    Title:  Sr. Vice ...  [Title, International Sales Date, ____________...   

                                                                                                   verbs  \
file               sentence                                                                                
inputs\ex10-11.txt \nEX-10\n2\nex10-11.txt\nCFC INTERNATIONAL, INC...             [DATED, amends, dated]   
                   1.                                                                                 []   
                   General Provisions\n----------------------\n\nA...           [remain, stated, stated]   
                   This\naddendum applies to CFC products B10EK bl...                          [applies]   
                   2.0  Distribution\n-----------------\n\nBuyer a...          [agrees, purchase, owned]   
...                                                                                                  ...   
inputs\ex10.txt    13.6 This Agreement is deemed to have been ente...         [deemed, entered, applied]   
                   13.7 In the event that a court of competent jur...  [holds, enforced, remain, remain]   
                   In Witness Whereof, the parties have caused thi...                 [caused, executed]   
                   BAXTER HEALTHCARE CORP             CFC INTERNAT...                                 []   
                   Purchasing                    Title:  Sr. Vice ...                       [Purchasing]   

                                                                                                    subjects  \
file               sentence                                                                                    
inputs\ex10-11.txt \nEX-10\n2\nex10-11.txt\nCFC INTERNATIONAL, INC...                        [INTERNATIONAL]   
                   1.                                                                                     []   
                   General Provisions\n----------------------\n\nA...                           [Provisions]   
                   This\naddendum applies to CFC products B10EK bl...                             [addendum]   
                   2.0  Distribution\n-----------------\n\nBuyer a...                         [Buyer, which]   
...                                                                                                      ...   
inputs\ex10.txt    13.6 This Agreement is deemed to have been ente...                                     []   
                   13.7 In the event that a court of competent jur...  [court, provisi